In [1]:
from pyspark.sql.session import SparkSession
import findspark
findspark.init()
from pyspark.sql.types import *
import pyspark.sql.functions as fn

In [2]:
spark=SparkSession.Builder().master("local[2]").getOrCreate()    # 创建spark实例对象
sc = spark.sparkContext

# 数据导入

In [3]:
# 读取CSV文件
# data=spark.read.csv("./data/武汉二手房.csv",schema=data_schema,header=True,encoding='gbk')  
data=spark.read.csv("./data/武汉二手房.csv",header=True,encoding='gbk')  
data.show(3,truncate=False)                  # truncate=False表示显示 PySpark 数据框的全列内容
data.printSchema()                           # 打印数据结构信息
data.count()

+----------------------------------------------------------+----+----+----+-------+----+------------+----------+---------------+------+------+-------------+----------+-----+
|标题                                                      |室数|厅数|卫数|面积   |朝向|楼层数      |建造时间  |小区           |行政区|街道  |门牌号       |单价      |总价 |
+----------------------------------------------------------+----+----+----+-------+----+------------+----------+---------------+------+------+-------------+----------+-----+
|正南 3室2厅 精装修                                        |3   |2   |2   |97.28㎡|南  |低层(共45层)|2019年建造|保利新武昌     |洪山  |白沙洲|张家湾路8号  |17270元/㎡|168万|
|急卖科普公园大华铂金郦府 纯商品房南北通透 自住装修随时看房|2   |2   |1   |88.79㎡|南北|低层(共34层)|2018年建造|大华铂金郦府   |青山  |余家头|旅大街21号   |18584元/㎡|165万|
|汉阳琴台大道江景住宅 3室2厅2卫 102m 约199万               |3   |2   |2   |102㎡  |南  |null        |null      |新世界汉江·云赫|硚口  |古田  |知音大道588号|19510元/㎡|199万|
+----------------------------------------------------------+----+----+----+-------+----+------------+----------+---

22680

# 数据清洗

## 重复值处理

In [4]:
data = data.dropDuplicates()                 # 去除重复行
data.count()                                 # 由于存在大量的重复挂牌销售数据，所以在去重后整体数据量并不大

752

在房屋销售中存在大量的重复挂牌销售数据，所以在去重后整体数据量并不大。这样的数据量实际上仅使用panda、numpy与sklearn工具，就可以实现上述问题的分析。
但是考虑到本文的初衷是pysark的使用实战，所以我们仍旧使用这一数据，但是还是基于pyspark进行数据分析。

## 数据格式转换

substring_index(str,delim,count):
    
    在分隔符 delim 出现计数之前从字符串 str 返回子字符串。 如果计数为正，则最终分隔符左侧的所有内容（从左开始计数）均为 返回。如果计数为负数，则最终分隔符右侧的每个（从右）返回。
    
substring(str,pos,len):
    
    子字符串从 pos 开始，当 str 是字符串类型或 返回字节数组的切片，该数组从字节中的 POS 开始，当 str 为二进制类型时长度为 len。

In [5]:
data=data.withColumn("面积",fn.substring_index(data["面积"],'㎡', 1))                 # 面积：去单位
data=data.withColumn("单价",fn.substring_index(data["单价"],'元/㎡', 1))              # 单价：去单位
data=data.withColumn("总价",fn.substring_index(data["总价"],'万', 1))                 # 总价：去单位
data=data.withColumn("建造时间",fn.substring(data["建造时间"],0,4))                   # 建造时间：去后缀
data=data.withColumn("楼层所属区域",fn.substring(data["楼层数"],0,2))                 # 楼层所属区域：提取数据
data=data.drop("楼层数")                                                              # 删除信息重复列

In [6]:
data.show(3)

+-------------------------------------+----+----+----+------+----+--------+----------+------+------+-------------+-----+----+------------+
|                                 标题|室数|厅数|卫数|  面积|朝向|建造时间|      小区|行政区|  街道|       门牌号| 单价|总价|楼层所属区域|
+-------------------------------------+----+----+----+------+----+--------+----------+------+------+-------------+-----+----+------------+
|      劲爆价 直降50万 捡漏 好楼层 ...|   3|   2|   1|129.13|  南|    2007|  汉口春天|  硚口|  古田|解放大道109号|12236| 158|        中层|
|   急卖单价低 仁和路 星桥苑 金科城...|   2|   2|   1|100.11|南北|    2001|星桥苑小区|  洪山|欢乐谷|友谊大道988号|11987| 120|        中层|
|八号线地铁口文昌路站旁珞珈雅苑精装...|   2|   2|   1| 89.41|南北|    2014|  珞珈雅苑|  洪山|新南湖|  书城路369号|16665| 149|        高层|
+-------------------------------------+----+----+----+------+----+--------+----------+------+------+-------------+-----+----+------------+
only showing top 3 rows



In [7]:
data.printSchema()                           # 打印数据结构信息

root
 |-- 标题: string (nullable = true)
 |-- 室数: string (nullable = true)
 |-- 厅数: string (nullable = true)
 |-- 卫数: string (nullable = true)
 |-- 面积: string (nullable = true)
 |-- 朝向: string (nullable = true)
 |-- 建造时间: string (nullable = true)
 |-- 小区: string (nullable = true)
 |-- 行政区: string (nullable = true)
 |-- 街道: string (nullable = true)
 |-- 门牌号: string (nullable = true)
 |-- 单价: string (nullable = true)
 |-- 总价: string (nullable = true)
 |-- 楼层所属区域: string (nullable = true)



## 缺失值处理

In [8]:
# 查看缺失值占比
data.agg(*[(1- (fn.count(c)/ fn.count('*'))).alias(c + 'miss_rate') for c in data.columns]).show(vertical=True)
# count('*') 表示统计列的所有行数  
# agg(*[]) 将该列表作为一组独立的参数传递给函数，可以理解为*arg

-RECORD 0-------------------------------------
 标题miss_rate         | 0.0                  
 室数miss_rate         | 0.0                  
 厅数miss_rate         | 0.0                  
 卫数miss_rate         | 0.0                  
 面积miss_rate         | 0.0                  
 朝向miss_rate         | 0.0                  
 建造时间miss_rate     | 0.039893617021276584 
 小区miss_rate         | 0.0                  
 行政区miss_rate       | 0.0                  
 街道miss_rate         | 0.0                  
 门牌号miss_rate       | 0.0                  
 单价miss_rate         | 0.0                  
 总价miss_rate         | 0.0                  
 楼层所属区域miss_rate | 0.039893617021276584 



In [9]:
# 查看缺失值个数
data.agg(*[(fn.count('*')-fn.count(c)).alias(c + 'miss_num') for c in data.columns]).show(vertical=True)

-RECORD 0-------------------
 标题miss_num         | 0   
 室数miss_num         | 0   
 厅数miss_num         | 0   
 卫数miss_num         | 0   
 面积miss_num         | 0   
 朝向miss_num         | 0   
 建造时间miss_num     | 30  
 小区miss_num         | 0   
 行政区miss_num       | 0   
 街道miss_num         | 0   
 门牌号miss_num       | 0   
 单价miss_num         | 0   
 总价miss_num         | 0   
 楼层所属区域miss_num | 30  



通过查询发现，在本文数据中仅有楼层所属区域、建造时间两个变量中存在少量的缺失值，缺失值占比为3.99%，可以考虑删除缺失值。但是在实际分析中考虑到去重之后数据量本身并不大(仅有752条)，所以我们这里对缺失值进行插值补齐。

    对于数值型变量建造时间，基于平均数进行插值；
    对于分类型变量所在楼层利用众数进行插值。

In [10]:
# 楼层所属区域频数统计
data.groupby("楼层所属区域").count().sort("count",ascending=False).show()              
# 楼层所属区域的众数是高层

+------------+-----+
|楼层所属区域|count|
+------------+-----+
|        高层|  281|
|        低层|  227|
|        中层|  206|
|        null|   30|
|         共5|    4|
|         共1|    4|
+------------+-----+



In [11]:
# 缺失值填充
year_mean=int(round(data.select(fn.mean('建造时间')).collect()[0][0],0))       # 建造时间的均值
floor_mode="高层"
data=data.fillna({'建造时间':year_mean,'楼层所属区域':floor_mode})

In [12]:
data.show(3)

+-------------------------------------+----+----+----+------+----+--------+----------+------+------+-------------+-----+----+------------+
|                                 标题|室数|厅数|卫数|  面积|朝向|建造时间|      小区|行政区|  街道|       门牌号| 单价|总价|楼层所属区域|
+-------------------------------------+----+----+----+------+----+--------+----------+------+------+-------------+-----+----+------------+
|      劲爆价 直降50万 捡漏 好楼层 ...|   3|   2|   1|129.13|  南|    2007|  汉口春天|  硚口|  古田|解放大道109号|12236| 158|        中层|
|   急卖单价低 仁和路 星桥苑 金科城...|   2|   2|   1|100.11|南北|    2001|星桥苑小区|  洪山|欢乐谷|友谊大道988号|11987| 120|        中层|
|八号线地铁口文昌路站旁珞珈雅苑精装...|   2|   2|   1| 89.41|南北|    2014|  珞珈雅苑|  洪山|新南湖|  书城路369号|16665| 149|        高层|
+-------------------------------------+----+----+----+------+----+--------+----------+------+------+-------------+-----+----+------------+
only showing top 3 rows



## 数据保存

将清洗后的数据导出保存

In [13]:
data.write.csv('./data/data.csv', header=True)        # 如果文件已经存在，会报错